# Connect to your workspace
Below steps taken from: [Microsoft docs: Deploy your ML model](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-and-where?tabs=python)


In [1]:
from azureml.core import Workspace, Dataset
ws = Workspace.from_config()

# (Create / obtain and) Set the model

    For now, this model will not actively be used. Instead, the score.py will download the model JIT.
    In later iterations, this code can be used to set the 'model' variable to an existing model in the workspace. At that time, make sure to update the score.py as well.

In [ ]:
import urllib.request
from azureml.core.model import Model

# Download model
urllib.request.urlretrieve("https://aka.ms/bidaf-9-model", "model.onnx")

# Register model
model = Model.register(ws, model_name="bidaf_onnx", model_path="./model.onnx")

# Create a web service around the model

- Create a runtime environment (using the conda configuration with any dependencies)
- Define a wrapping script to act as the initialization and entry point for any incoming request (score.py)
- Create a local service (on the associated compute) to build the container and test the results

In [26]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig

env = Environment(name="project_environment").from_conda_specification(name = "lease-abstract", 
                              file_path = "../config/inference-conda.yml")

inference_config = InferenceConfig(
    environment=env,
    source_directory="../",
    entry_script="./score.py",
)

In [27]:
from azureml.core.webservice import LocalWebservice

deployment_config = LocalWebservice.deploy_configuration(port=10002)

In [ ]:
service = Model.deploy(
    ws,
    "leaseabstractservice",
    [model],
    inference_config,
    deployment_config,
    overwrite=True,
)
service.wait_for_deployment(show_output=True)

In [ ]:
print(service.get_logs())

## Test the local service

In [ ]:
import requests
import json

uri = service.scoring_uri
requests.get("http://localhost:10002")
headers = {"Content-Type": "application/json"}
data = {
    "id": "123456789",
    "content": "The quick brown fox jumped over the lazy dog.",
}
data = json.dumps(data)
response = requests.post(uri, data=data, headers=headers)
print(response.json())

# Deploy the model to the target compute
In this demo, we will use Azure Container Instances (as they're easier to setup and appropriate for initial experimentation). In production / at scale, the use of AKS is recommended.

The document linked at the beginning of this article describes how to use alternative deployment options.

In [ ]:
from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(cpu_cores=2, memory_gb=4, auth_enabled=True)

service = Model.deploy(ws, "leaseabstractservice", [model], inference_config, deployment_config, overwrite=True)

service.wait_for_deployment(show_output=True)

In [ ]:
print(service.get_logs())

## Invoke the Aci-hosted web service.

In [ ]:
import requests
import json
from azureml.core import Webservice

service = Webservice(workspace=ws, name="leaseabstractservice")
scoring_uri = service.scoring_uri

# If the service is authenticated, set the key or token
key, _ = service.get_keys()

# Set the appropriate headers
headers = {"Content-Type": "application/json"}
headers["Authorization"] = f"Bearer {key}"

# Make the request and display the response and logs
data = {
    "id": "MyContent",
    "content": "The quick brown fox jumped over the lazy dog.",
}
data = json.dumps(data)
resp = requests.post(scoring_uri, data=data, headers=headers)
print(resp.text)